In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = 30
import os
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import datetime
import pymysql
import time
from datetime import date
import openpyxl
import glob
import shutil


print("Importado con éxito")

In [13]:
try:
    connection = pymysql.connect(host="192.168.100.215",user="root", passwd="sqladmin", db="siogranos", port=3316)
    cursor = connection.cursor()
    data = pd.read_sql_query (f"SELECT * FROM data", connection)
    connection.commit()
    connection.close()
    print("La base se ha cargado correctamente")
except:
    print("No se encontró la base de datos del SIO-GRANOS")

C:\Users\frolotti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


No se encontró la base de datos del SIO-GRANOS


In [17]:
# Esta celda es para arreglar el formato de la fecha de la base de datos, ya que Python tiene por defecto fechas en inglés:

data = data[data['FECHA CONCERTACION'].str.contains('CÓRDOBA')==False]

data.info()



AttributeError: Can only use .str accessor with string values!

In [18]:
#ordenamos la base según la fecha de operacion:
data=data.sort_values(by="FECHA CONCERTACION")
# Tdata.head()

,FECHA OPERACION,FECHA CONCERTACION,OPERACION,TIPO,PRECIO,PRODUCTO,CANT. (TN),CALIDAD,CALIDAD ADICIONAL,PROCEDENCIA PCIA,PROCEDENCIA LOCALID.,PRECIO/TN MONEDA,PRECIO/TN MONTO,LUGAR ENTREGA,FECHA ENTR. DESDE,FECHA ENTR. HASTA,CONDICION PAGO,ES FINAL,COSECHA,NRO INSTANCIA OPERACION,Unnamed: 20
578529,2021-08-26,2002-08-01,Contrato,Compraventa,Precio Hecho,MAIZ,60.37,Otra,nan,CÓRDOBA,CALASUYA,$,17860.0,Zona 11/En destino,01/08/2021,30/08/2021,Contra entrega,SI,COSECHA 20/21,1.0,nan
1288449,2022-12-29,2004-12-28,Contrato,Compraventa,Precio Hecho,SOJA,200.0,Fábrica,nan,CÓRDOBA,ADELIA MARIA,U$S,149.48,Rosario N/En destino,01/04/2005,31/05/2005,Contra entrega,SI,COSECHA 04/05,1.0,nan
1288447,2022-12-29,2004-12-28,Contrato,Compraventa,Fijar Precio/Mercado Comprador,SOJA,300.0,Fábrica,nan,CÓRDOBA,VIAMONTE,0,0.0,Zona 8/En destino,01/03/2005,30/05/2005,A plazo,SI,COSECHA 04/05,1.0,nan
1288443,2022-12-29,2005-01-13,Contrato,Compraventa,Precio Hecho,SOJA,30.0,Fábrica,nan,CÓRDOBA,JESUS MARIA,U$S,143.0,Rosario N/En destino,01/03/2005,30/05/2005,Contra entrega,SI,COSECHA 04/05,1.0,nan
1288448,2022-12-29,2005-01-23,Contrato,Compraventa,Fijar Precio/Mercado Comprador,SOJA,700.0,Fábrica,nan,CÓRDOBA,VIAMONTE,0,0.0,Zona 8/En destino,01/03/2005,30/05/2005,A plazo,SI,COSECHA 04/05,1.0,nan


In [27]:
# Reemplazamos todos los "A Fijar" diferentes por un sólo tipo de "A Fijar":

data.loc[data.PRECIO.isin(['Fijar Precio/Prec. Cam.','Fijar Precio/Mercado Comprador',
'Fijar Precio/Otros', 'Fijar Precio/M. T. País', 'Fijar Precio','Fijar Precio/Fas teórico','Fijar Precio/PR R. Conj.',
 'Fijar Precio/M. T. Ext.']),"PRECIO"] = "A Fijar"


# Cambiamos la condición de pago a "A Plazo" y a "Contraentrega":
data.loc[~data["CONDICION PAGO"].isin(['A plazo']),"CONDICION PAGO"] = "Contraentrega"



# Creo una dummy para ver si el comprador es final con destino a Córdoba, o si es una venta al exterior de Córdoba (final o no).
# Si la venta es a Córdoba pero NO es a un comprador final, no quiero tenerla en cuenta, porque 
# esa mercadería se volvería a comercializar y la contabilizaría dos veces. Si la mercadería sale de
# Córdoba, no me interesa si es final o no, porque no vuelve a aparecer en la base y ya salió de la provincia
# (ya que sólo tengo datos que provienen de Córdoba)
# DUMMY = "No usar", no me interesa. DUMMY = "Usar", la voy a usar en el análisis

cordoba = ["Cordoba/En destino","Zona 11/En destino","Zona 8/En destino","Zona 9/En destino",
          "Cordoba/En origen","Zona 11/En origen","Zona 8/En origen","Zona 9/En origen"]

# Indico que si no es final y tiene como destino a Córdoba me agregue que esta operacion no voy a usarla, de lo contrario sí
# voy a usarla:

data["DUMMY"] = np.where((data['ES FINAL']=="NO") & (data["LUGAR ENTREGA"].isin(cordoba)), 'No usar', 'Usar')



data


,FECHA OPERACION,FECHA CONCERTACION,OPERACION,TIPO,PRECIO,PRODUCTO,CANT. (TN),CALIDAD,CALIDAD ADICIONAL,PROCEDENCIA PCIA,PROCEDENCIA LOCALID.,PRECIO/TN MONEDA,PRECIO/TN MONTO,LUGAR ENTREGA,FECHA ENTR. DESDE,FECHA ENTR. HASTA,CONDICION PAGO,ES FINAL,COSECHA,NRO INSTANCIA OPERACION,Unnamed: 20,DUMMY
578529,2021-08-26,2002-08-01,Contrato,Compraventa,Precio Hecho,MAIZ,60.37,Otra,nan,CÓRDOBA,CALASUYA,$,17860.0,Zona 11/En destino,01/08/2021,30/08/2021,Contraentrega,SI,COSECHA 20/21,1.0,nan,Usar
1288449,2022-12-29,2004-12-28,Contrato,Compraventa,Precio Hecho,SOJA,200.0,Fábrica,nan,CÓRDOBA,ADELIA MARIA,U$S,149.48,Rosario N/En destino,01/04/2005,31/05/2005,Contraentrega,SI,COSECHA 04/05,1.0,nan,Usar
1288447,2022-12-29,2004-12-28,Contrato,Compraventa,A Fijar,SOJA,300.0,Fábrica,nan,CÓRDOBA,VIAMONTE,0,0.0,Zona 8/En destino,01/03/2005,30/05/2005,A plazo,SI,COSECHA 04/05,1.0,nan,Usar
1288443,2022-12-29,2005-01-13,Contrato,Compraventa,Precio Hecho,SOJA,30.0,Fábrica,nan,CÓRDOBA,JESUS MARIA,U$S,143.0,Rosario N/En destino,01/03/2005,30/05/2005,Contraentrega,SI,COSECHA 04/05,1.0,nan,Usar
1288448,2022-12-29,2005-01-23,Contrato,Compraventa,A Fijar,SOJA,700.0,Fábrica,nan,CÓRDOBA,VIAMONTE,0,0.0,Zona 8/En destino,01/03/2005,30/05/2005,A plazo,SI,COSECHA 04/05,1.0,nan,Usar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853485,2023-02-15,2023-02-15,Contrato,Compraventa,A Fijar,MAIZ,40919.26,Cámara,nan,CÓRDOBA,JESUS MARIA,0,0.0,Cordoba/En destino,01/07/2023,31/07/2023,Contraentrega,SI,COSECHA 22/23,1,None,Usar
1853484,2023-02-15,2023-02-15,Contrato,Compraventa,Precio Hecho,MAIZ,215.91,Cámara,nan,CÓRDOBA,CHALACEA,$,49970.0,Cordoba/En destino,15/02/2023,17/03/2023,Contraentrega,SI,COSECHA 21/22,1,None,Usar
1853524,2023-02-15,2023-02-15,Contrato,Compraventa,Precio Hecho,MAIZ,150.0,Cámara,nan,CÓRDOBA,INRIVILLE,U$S,260.0,Rosario S/En destino,15/02/2023,17/03/2023,Contraentrega,NO,COSECHA 21/22,1,None,No usar
1853489,2023-02-15,2023-02-15,Contrato,Compraventa,A Fijar,MAIZ,54446.52,Cámara,nan,CÓRDOBA,JESUS MARIA,0,0.0,Cordoba/En destino,01/07/2023,31/07/2023,Contraentrega,SI,COSECHA 22/23,1,None,Usar


In [30]:
# La cosecha no se encuentra armonizada, muchos agregan sólo el año de cosecha, mientras
# que otros escriben los dos años de la campaña. Por ejemplo, algunos escriben "COSECHA 2021" y otros "COSECHA 20/21",
# que es escencialmente lo mismo. Normalizo eso:

data = data[data['COSECHA'].str.contains('SI')==False]
data = data[data['COSECHA'].str.contains('an')==False]

#Elimino los datos donde existe un error en la COSECHA, y se la ha cargado como "0":
data[data.COSECHA==0].count()
data = data[data.COSECHA!=0]

data.COSECHA = data.COSECHA.map(lambda x: x if x.find("/")>=0 else "COSECHA "+str(int(x[-2:])-1).zfill(2)+"/"+x[-2:].zfill(2))
data.COSECHA = data.COSECHA.map(lambda x: x.replace("-1", "99"))
data.COSECHA.unique()

array(['COSECHA 20/21', 'COSECHA 04/05', 'COSECHA 03/04', 'COSECHA 11/12',
       'COSECHA 12/13', 'COSECHA 15/16', 'COSECHA 99/00', 'COSECHA 13/14',
       'COSECHA 14/15', 'COSECHA 16/17', 'COSECHA 08/09', 'COSECHA 09/10',
       'COSECHA 10/11', 'COSECHA 06/07', 'COSECHA 07/08', 'COSECHA 17/18',
       'COSECHA 05/06', 'COSECHA 01/02', 'COSECHA 18/19', 'COSECHA 00/01',
       'COSECHA 19/20', 'COSECHA 02/03', 'COSECHA 21/22', 'COSECHA 22/23',
       'COSECHA 23/24'], dtype=object)

In [33]:
cosecha = data['COSECHA'].unique()
producto = data['PRODUCTO'].unique()

In [42]:
productos = {}

for i in producto:
    productos[i] = data[data['PRODUCTO'].str.contains(i)==True]
    
    
productos


C:\Users\frolotti\AppData\Local\Temp\ipykernel_18744\682826638.py:4: FutureWarning: Dropping invalid columns in DataFrameGroupBy.add is deprecated. In a future version, a TypeError will be raised. Before calling .add, select only columns which should be valid for the function.
  productos[i] = data[data['PRODUCTO'].str.contains(i)==True].groupby(['FECHA CONCERTACION', 'OPERACION', 'COSECHA']).sum()
C:\Users\frolotti\AppData\Local\Temp\ipykernel_18744\682826638.py:4: FutureWarning: Dropping invalid columns in DataFrameGroupBy.add is deprecated. In a future version, a TypeError will be raised. Before calling .add, select only columns which should be valid for the function.
  productos[i] = data[data['PRODUCTO'].str.contains(i)==True].groupby(['FECHA CONCERTACION', 'OPERACION', 'COSECHA']).sum()
C:\Users\frolotti\AppData\Local\Temp\ipykernel_18744\682826638.py:4: FutureWarning: Dropping invalid columns in DataFrameGroupBy.add is deprecated. In a future version, a TypeError will be raised.

{'MAIZ':                                                                                              TIPO  \
 FECHA CONCERTACION OPERACION     COSECHA                                                            
 2002-08-01         Contrato      COSECHA 20/21                                        Compraventa   
 2013-01-05         Contrato      COSECHA 11/12                          CanjeCanjeCanjeCanjeCanje   
 2013-08-13         Contrato      COSECHA 12/13                                              Canje   
 2013-10-12         Contrato      COSECHA 12/13                                         CanjeCanje   
 2014-01-02         Contrato      COSECHA 99/00                                              Canje   
 ...                                                                                           ...   
 2023-02-14         Rectificación COSECHA 21/22       CompraventaCompraventaCompraventaCompraventa   
 2023-02-15         Contrato      COSECHA 20/21       CompraventaCompraven

C:\Users\frolotti\AppData\Local\Temp\ipykernel_18744\1861848359.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.add is deprecated. In a future version, a TypeError will be raised. Before calling .add, select only columns which should be valid for the function.
  tn_cosecha = data.groupby(['FECHA CONCERTACION', 'PRODUCTO','COSECHA', 'OPERACION', 'CANT. (TN)']).sum()


TIPO  \
FECHA CONCERTACION PRODUCTO  COSECHA       OPERACION CANT. (TN)                           
2002-08-01         MAIZ      COSECHA 20/21 Contrato  60.37                  Compraventa   
2004-12-28         SOJA      COSECHA 04/05 Contrato  200.0                  Compraventa   
                                                     300.0                  Compraventa   
2005-01-13         SOJA      COSECHA 04/05 Contrato  30.0                   Compraventa   
2005-01-23         SOJA      COSECHA 04/05 Contrato  700.0                  Compraventa   
...                                                                                 ...   
2023-02-15         TRIGO PAN COSECHA 22/23 Contrato  30.0        CompraventaCompraventa   
                                                     300.0                  Compraventa   
                                                     520.0                  Compraventa   
                                                     60.0                   Compraventa   
                                                     67.0                   Compraventa   

                                                                                         PRECIO  \
FECHA CONCERTACION PRODUCTO  COSECHA       OPERACION CANT. (TN)                                   
2002-08-01         MAIZ      COSECHA 20/21 Contrato  60.37                         Precio Hecho   
2004-12-28         SOJA      COSECHA 04/05 Contrato  200.0                         Precio Hecho   
                                                     300.0       Fijar Precio/Mercado Comprador   
2005-01-13         SOJA      COSECHA 04/05 Contrato  30.0                          Precio Hecho   
2005-01-23         SOJA      COSECHA 04/05 Contrato  700.0       Fijar Precio/Mercado Comprador   
...                                                                                         ...   
2023-02-15         TRIGO PAN COSECHA 22/23 Contrato  30.0              Precio HechoPrecio Hecho   
                                                     300.0                         Precio Hecho   
                                                     520.0                         Precio Hecho   
                                                     60.0                    Fijar Precio/Otros   
                                                     67.0                          Precio Hecho   

                                                                       CALIDAD  \
FECHA CONCERTACION PRODUCTO  COSECHA       OPERACION CANT. (TN)                  
2002-08-01         MAIZ      COSECHA 20/21 Contrato  60.37                Otra   
2004-12-28         SOJA      COSECHA 04/05 Contrato  200.0             Fábrica   
                                                     300.0             Fábrica   
2005-01-13         SOJA      COSECHA 04/05 Contrato  30.0              Fábrica   
2005-01-23         SOJA      COSECHA 04/05 Contrato  700.0             Fábrica   
...                                                                        ...   
2023-02-15         TRIGO PAN COSECHA 22/23 Contrato  30.0        CámaraArt. 12   
                                                     300.0              Cámara   
                                                     520.0              Cámara   
                                                     60.0               Cámara   
                                                     67.0               Cámara   

                                                                CALIDAD ADICIONAL  \
FECHA CONCERTACION PRODUCTO  COSECHA       OPERACION CANT. (TN)                     
2002-08-01         MAIZ      COSECHA 20/21 Contrato  60.37                    nan   
2004-12-28         SOJA      COSECHA 04/05 Contrato  200.0                    nan   
                                                     300.0                    nan   
2005-01-13         SOJA      COSECHA 04/05 Contrato  30.0                     nan   
2005-01-23         SOJA      COSEC

In [25]:
tn_cosecha.pivot('FECHA CONCERTACION', 'OPERACION', 'CANT. (TN)')

KeyError: 'FECHA CONCERTACION'